In [1]:
import time
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import re
import requests

In [2]:
# driver = webdriver.Chrome()

In [3]:
# https://www.kmdb.or.kr/db/kor/detail/movie/F/47808

# page_url = 'https://www.kmdb.or.kr/db/have/detailSearch/imageSearch'
# driver.get(page_url)
# driver = webdriver.Chrome()
# index = 47808
# for i in range(50):
#     page_url = f'https://www.kmdb.or.kr/db/kor/detail/movie/F/{index + i}'
#     driver.get(page_url)
#     time.sleep(1)

In [4]:
import requests
from bs4 import BeautifulSoup
import re
import time

genre_mapping = {
    "드라마": 1 << 43,
    "첩보": 1 << 42,
    "옴니버스": 1 << 41,
    "뮤직": 1 << 40,
    "동성애": 1 << 39,
    "로드무비": 1 << 38,
    "아동": 1 << 37,
    "하이틴(고교)": 1 << 36,
    "전기": 1 << 35,
    "청춘영화": 1 << 34,
    "재난": 1 << 33,
    "문예": 1 << 32,
    "연쇄극": 1 << 31,
    "신파": 1 << 30,
    "활극": 1 << 29,
    "반공/분단": 1 << 28,
    "군사": 1 << 27,
    "계몽": 1 << 26,
    "사회물(경향)": 1 << 25,
    "스포츠": 1 << 24,
    "합작(번안물)": 1 << 23,
    "종교": 1 << 22,
    "무협": 1 << 21,
    "미스터리": 1 << 20,
    "SF": 1 << 19,
    "코메디": 1 << 18,
    "느와르": 1 << 17,
    "액션": 1 << 16,
    "범죄": 1 << 15,
    "어드벤처": 1 << 14,
    "가족": 1 << 13,
    "에로": 1 << 12,
    "멜로/로맨스": 1 << 11,
    '멜로드라마': 1 << 11,
    "공포": 1 << 10,
    "뮤지컬": 1 << 9,
    "시대극/사극": 1 << 8,
    '실험': 1 << 7,
    '스릴러': 1 << 5,
    '서부': 1 << 4,
    '전쟁': 1 << 3,
    '갱스터': 1 << 2,
    '판타지': 1 << 1,
    '해양액션': 1 << 0
}

def genreIncode(strArr: list) -> int:
    """
    - 채연이가 만든 함수
    - strArr : 영화 제목 리스트
    - return : 제목 값 더한거
    """
    
    if len(strArr) == 0:
        return 0 
    sum = 0
    for i in strArr:
        print(i)
        if i.startswith('#'):
            i = i.replace('#', '')
        if i in genre_mapping:
            print(i, genre_mapping[i])
            sum += genre_mapping[i]
    return sum

def titleList(fileName : str):
    
    movieNum = int(fileName.split('_')[1].split('.')[0]) #int(movie)
    titleListGenre = []
    for k, v in genre_mapping.items():
        titleListGenre.append(1 if v & movieNum else 0)
        
    return titleListGenre

def get_image_data_list(folder_path, target_size=(400, 600)):
    # 지원하는 이미지 확장자
    image_extensions = ('.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff')
    
    # 이미지 데이터 목록을 저장할 리스트
    image_data_list = []
    image_genre_list = []
    # 폴더 내의 모든 파일을 순회
    for filename in os.listdir(folder_path):
        # 파일의 경로 생성
        file_path = os.path.join(folder_path, filename)
        
        # 파일이 이미지 확장자 중 하나인지 확인
        if filename.lower().endswith(image_extensions) and os.path.isfile(file_path):
            # 이미지를 열고 지정된 크기로 리사이즈
            print(file_path, os.path.basename(file_path))
            img = Image.open(file_path)
            img_resized = img.resize(target_size)  # 크기 조정
            img_data = np.array(img_resized)  # 이미지를 NumPy 배열로 변환
            image_data_list.append(img_data)
            image_genre_list.append(titleList(os.path.basename(file_path)))

    return (image_data_list, image_genre_list)

def save_image(img_url, file_name, save_folder):
    # 이미지 저장 폴더가 없으면 생성
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    
    # 이미지 다운로드
    img_response = requests.get(img_url)
    
    # 응답이 성공적인지 확인
    if img_response.status_code == 200:
        file_path = os.path.join(save_folder, file_name)
        with open(file_path, 'wb') as f:
            f.write(img_response.content)
        print(f"이미지 저장 완료: {file_path}")
    else:
        print(f"이미지 다운로드 실패: {img_url}")
        
def fetch_url_with_retries(url, max_retries=5, delay=2):
    for attempt in range(max_retries):
        try:
            response = requests.get(url)
            response.raise_for_status()  # 상태 코드가 200이 아닐 경우 예외 발생
            return response  # 요청 성공 시 응답 반환
        except requests.exceptions.RequestException as e:
            print(f"요청 실패: {e}")
            if attempt < max_retries - 1:  # 마지막 시도가 아닐 경우
                print(f"{delay}초 후에 재시도합니다...")
                time.sleep(delay)  # 지연 후 재시도
            else:
                print("최대 재시도 횟수에 도달했습니다.")
                return None  # 실패 시 None 반환
        
# URL 설정
# url = "https://example.com"
index = 45300
for i in range(20000):
    time.sleep(0.5)
    page_url = f'https://www.kmdb.or.kr/db/kor/detail/movie/F/{index + i}'
    # response = requests.get(page_url)
    response = fetch_url_with_retries(page_url)
    if response is not None:
        print(f"성공적으로 가져온 페이지: {page_url}")
        # 여기서 response.text 등으로 추가 작업을 할 수 있습니다.
    else:
        print(f"페이지 가져오기 실패: {page_url}. 다음 페이지로 넘어갑니다.")
        continue  # 다음 페이지로 넘어감

    # 웹 페이지 요청
    # response = requests.get(url)

    # 상태 코드 확인
    if response.status_code == 200:
        # HTML 파싱
        soup = BeautifulSoup(response.content, 'html.parser')
        # /html/body/div[2]/div[5]/div[2]/form/section/div[1]/div[2]/div[1]/div[1]
        # 웹 페이지의 제목 가져오기
        div_tag = soup.find('div', class_='nam')
        div_tag = div_tag.find('div', class_='na1')
        # title = soup.title.string
        print(f"페이지 제목: {div_tag.text}", index + i)

        img_url = soup.find('div', class_='mProfile type2')
        img_url = img_url.find('div', class_='mImg1')
        img_url = img_url.find('span')
        img_url = img_url.get('style')
        img_url = re.search(r"url\('(.*?)'\)", img_url).group(1)
        if not img_url.startswith('http'):
            continue
        genre_arr = soup.find('div', class_='mList12')
        if not genre_arr:
            continue
        genre_arr = genre_arr.find('dd', class_='mTag1 gray')
        if not genre_arr:
            continue
        genre_arr = genre_arr.find_all('a')
        genre_arr2 = []
        for j in genre_arr:
            genre_arr2.append(j.text)
        
        print(f"img_url: {img_url}", type(img_url), index + i, div_tag.text)
        # for i in genre_arr : List[NavigableString]:
        # for i in genre_arr :
        #      print(i.text.replace('#', ''))
        print('genre_arr : ', len(genre_arr))
        # 제목_장르배열코드.jpg
        title = div_tag.text.strip().replace(":", '')
        title = re.sub(r'[^A-Za-z0-9\s가-힣]', '', title)  # 특수 문자를 제거하고 한글 포함
        genre_arr_code = genreIncode(genre_arr2)
        file_name = f'{title}_{genre_arr_code}.jpg'
        print('file_name : ', file_name)
        
        save_image(img_url, file_name, r'G:\내 드라이브\project_poster\x_train')
        
        # # # 이미지 저장
        # image_url = image.get_attribute('href')
        # image_name = f"{genre_labels}.jpg"  # 파일명 지정
        # driver.get(image_url)

        # with open(os.path.join(download_folder, image_name), 'wb') as file:
        #     file.write(driver.page_source.encode('utf-8'))

        # driver.back()
        # time.sleep(1)
        
        
        
        
        
        # # 모든 링크 가져오기
        # links = soup.find_all('a')

        # print("링크:")
        # for link in links:
        #     href = link.get('href')
        #     text = link.string
        #     print(f"텍스트: {text}, URL: {href}")
    else:
        print(f"웹 페이지를 가져오는 데 실패했습니다. 상태 코드: {response.status_code}")


성공적으로 가져온 페이지: https://www.kmdb.or.kr/db/kor/detail/movie/F/45300
페이지 제목: 
						촬영당한 대실금 
						
						
					 45300
성공적으로 가져온 페이지: https://www.kmdb.or.kr/db/kor/detail/movie/F/45301
페이지 제목: 
						지옥의 분노 
						
						
					 45301
성공적으로 가져온 페이지: https://www.kmdb.or.kr/db/kor/detail/movie/F/45302
페이지 제목: 
						핑크커튼 방의 사촌누나 
						
						
					 45302
성공적으로 가져온 페이지: https://www.kmdb.or.kr/db/kor/detail/movie/F/45303
페이지 제목: 
						다다미방의 친구엄마 
						
						
					 45303
성공적으로 가져온 페이지: https://www.kmdb.or.kr/db/kor/detail/movie/F/45304
페이지 제목: 
						전망 좋은 집: 맛있는 섹스 
						
						
					 45304
성공적으로 가져온 페이지: https://www.kmdb.or.kr/db/kor/detail/movie/F/45305
페이지 제목: 
						긴머리의 섹시한 누나 
						
						
					 45305
성공적으로 가져온 페이지: https://www.kmdb.or.kr/db/kor/detail/movie/F/45306
페이지 제목: 
						마지막 게임 
						
						
					 45306
성공적으로 가져온 페이지: https://www.kmdb.or.kr/db/kor/detail/movie/F/45307
페이지 제목: 
						핫팬츠의 사촌누나 
						
						
					 45307
성공적으로 가져온 페이지: https://www.kmdb.or.kr/db/kor/detail